# Google Shopping API

In [ ]:
!pip install requests pandas

In [ ]:
# Import libraries
import json
import requests
import pandas as pd

In [ ]:
# Oxylabs test API
payload = {  
  'query': 'truss',
  'contry': 'BR',
  'language': 'pt',
}

response = requests.request(
  'POST',
  'hts://realtime.oxylabs.io/v1/queries',
)